### Import Libraries

In [1]:
import numpy as np
import re
import pickle
import pandas as pd 
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files

### Load Dataset

In [2]:
dataset = pd.read_table('SMSSPamCollection', header=None, encoding='utf-8')

C:\Users\prasad\Anaconda3\envs\tensorflow9\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


### Feature Engineering

In [3]:
sentiments=[]
text=[]
for _ in dataset[0].values:
    sentiments.append(_)
for _ in dataset[1].values:
    text.append(_)
    
sentiments = pd.DataFrame(sentiments)
text = pd.DataFrame(text)

# change words to lower case
reviews = text[0].str.lower()
# Replace email addresses with string 'emailaddress'
reviews = reviews.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$','emailaddress')
# Remove punctuation
reviews = reviews.str.replace(r'[^\w\d\s]', ' ')

#### Create Bag of Words Model

Unlike humans, machine cannot understand raw text. Hence need to convert text into corresponding numerical form.<br>
Bag of Words can be approached for the task.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 2000, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
reviews = vectorizer.fit_transform(reviews).toarray()

#### Create Tf-Idf Model

Bag of Words can convert text into numbers via assigning scores for only one particular document.<br>
Tf-Idf overcome this drawback by depicting frequency score of occurence in other documents as well.

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
reviews = transformer.fit_transform(processed).toarray()

In [6]:
#Can also skip above two steps with this one step by creating Tf-Idf model directly w/o BoW model
#from sklearn.feature_extraction.text import TfidfVectorizer

#vectorizer = TfidfVectorizer(max_features = 2000, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
#reviews = vectorizer.fit_transform(reviews).toarray()

#### Splitting Dataset

In [7]:
from sklearn.model_selection import train_test_split
reviews_train, reviews_test, sent_train, sent_test = train_test_split(reviews, sentiments, test_size = 0.20, random_state = 0)

### Training

In [8]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(reviews_train,reviews_train)

C:\Users\prasad\Anaconda3\envs\tensorflow9\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\prasad\Anaconda3\envs\tensorflow9\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

### Prediction

In [9]:
reviews_pred = classifier.predict(reviews_test)

In [20]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
results = confusion_matrix(sent_test, sent_pred) 
print('Confusion Matrix :')
print(results) 
print('Report : ')
print(classification_report(sent_test, reviews_pred))
print('Accuracy Score :',accuracy_score(sent_test, reviews_pred)*100) 

Confusion Matrix :
[[953   2]
 [ 30 130]]
Report : 
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98       955
        spam       0.98      0.81      0.89       160

    accuracy                           0.97      1115
   macro avg       0.98      0.91      0.94      1115
weighted avg       0.97      0.97      0.97      1115

Accuracy Score : 97.13004484304932
